In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model

In [ ]:
df=pd.read_csv('/content/MY_Final_Dataset_csv.csv')

In [ ]:
# Splitting dataset into features and label
X=df.drop('Label', axis =1)
y = df['Label']

# Splitting the dataset into the training set and the test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
# scale data
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)

In [ ]:
# AutoEncoder Model Preparation
n_inputs = X.shape[1]
# define encoder
input_data_shape= Input(shape=(n_inputs,))
# encoder level 1
encoder= Dense(n_inputs*2)(input_data_shape)
encoder = BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# encoder level 2
encoder= Dense(n_inputs)(encoder)
encoder= BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(encoder)
# define decoder, level 1
decoder = Dense(n_inputs)(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)
# decoder level 2
decoder = Dense(n_inputs*2)(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)

In [ ]:
# output layer
output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
model = Model(inputs=input_data_shape, outputs=output)
# compile autoencoder model
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 66)]              0         
                                                                 
 dense (Dense)               (None, 132)               8844      
                                                                 
 batch_normalization (BatchN  (None, 132)              528       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 132)               0         
                                                                 
 dense_1 (Dense)             (None, 66)                8778      
                                                                 
 batch_normalization_1 (Batc  (None, 66)               264       
 hNormalization)                                             

In [ ]:
# fit the autoencoder model to reconstruct input
history = model.fit(X_train, y_train, epochs=500, batch_size=16, verbose=2, validation_data=(X_test,y_test))

Epoch 1/500
651/651 - 2s - loss: 0.0014 - val_loss: 0.0097 - 2s/epoch - 3ms/step
Epoch 2/500
651/651 - 2s - loss: 0.0012 - val_loss: 0.0131 - 2s/epoch - 3ms/step
Epoch 3/500
651/651 - 2s - loss: 0.0012 - val_loss: 8.0349e-04 - 2s/epoch - 3ms/step
Epoch 4/500
651/651 - 2s - loss: 5.5904e-04 - val_loss: 0.0011 - 2s/epoch - 3ms/step
Epoch 5/500
651/651 - 2s - loss: 4.4793e-04 - val_loss: 0.0014 - 2s/epoch - 3ms/step
Epoch 6/500
651/651 - 2s - loss: 0.0010 - val_loss: 0.0043 - 2s/epoch - 3ms/step
Epoch 7/500
651/651 - 2s - loss: 3.4311e-04 - val_loss: 4.5398e-04 - 2s/epoch - 3ms/step
Epoch 8/500
651/651 - 2s - loss: 0.0015 - val_loss: 0.0022 - 2s/epoch - 3ms/step
Epoch 9/500
651/651 - 2s - loss: 7.5609e-04 - val_loss: 0.0030 - 2s/epoch - 3ms/step
Epoch 10/500
651/651 - 2s - loss: 5.6331e-04 - val_loss: 0.0039 - 2s/epoch - 3ms/step
Epoch 11/500
651/651 - 2s - loss: 5.0539e-04 - val_loss: 7.3608e-04 - 2s/epoch - 3ms/step
Epoch 12/500
651/651 - 2s - loss: 3.9595e-04 - val_loss: 0.0040 - 2s/ep

In [ ]:
# define an encoder model (without the decoder)
encoder = Model(inputs=input_data_shape, outputs=bottleneck)
# save the encoder to file
encoder.save('encoder.h5')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)
# define model
model = LogisticRegression()
# fit model on training set
model.fit(X_train, y_train)
# make prediction on test set
ypred = model.predict(X_test)
# calculate accuracy
acc = accuracy_score(y_test, ypred)
print(acc)

0.9873125720876585


In [ ]:
#Compressing the input data using Encoder Model and fitting it on the Logistic Regression model.
# load the model from file
encoder = load_model('encoder.h5')

# encode the train data
X_train_encode = encoder.predict(X_train)
# encode the test data
X_test_encode = encoder.predict(X_test)
# define the model
model = LogisticRegression(max_iter=1000)
# fit the model on the training set
model.fit(X_train_encode, y_train)
# make predictions on the test set
yhat = model.predict(X_test_encode)
# calculate classification accuracy
acc = accuracy_score(y_test, yhat)
print(acc)

0.9988465974625144


In [ ]:
from xgboost import XGBClassifier


In [ ]:
#Compressing the input data using Encoder Model and fitting it on the Logistic Regression model.
# load the model from file
encoder = load_model('encoder.h5')

# encode the train data
X_train_encode = encoder.predict(X_train)
# encode the test data
X_test_encode = encoder.predict(X_test)
# define the model
model = XGBClassifier(max_iter=1000)
# fit the model on the training set
model.fit(X_train_encode, y_train)
# make predictions on the test set
yhat = model.predict(X_test_encode)
# calculate classification accuracy
acc = accuracy_score(y_test, yhat)
print(acc)

0.9988465974625144
